In [7]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 41kB/s 
     |████████████████████████████████| 3.8MB 43.1MB/s 
     |████████████████████████████████| 512kB 51.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=682de8fbbc7fc1f2fe81ffc15e322c6d989f4e498acc01e19b59a79e35b058f4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.

In [1]:
!git clone https://github.com/Microsoft/Recommenders

Cloning into 'Recommenders'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21630 (delta 5), reused 2 (delta 0), pack-reused 21616
Receiving objects: 100% (21630/21630), 29.91 MiB | 10.98 MiB/s, done.
Resolving deltas: 100% (13855/13855), done.
/content/Recommenders


In [2]:
%cd Recommenders

/content/Recommenders


In [0]:
#Move data to Recommenders and execute in 'Recommenders'
%cp ../LD_train.csv .
%cp ../LD_test.csv .
%cp ../NY_train.csv .
%cp ../NY_test.csv .

In [21]:
import sys
sys.path.append("../../")
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, ndcg_score
from reco_utils.common.timer import Timer
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset import movielens
from reco_utils.common.notebook_utils import is_jupyter
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))



System version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
Pandas version: 1.0.3
Tensorflow version: 1.15.0


In [0]:
# top k items to recommend
TOP_K = 5

# Select Dataset 
train_data = pd.read_csv('LD_train.csv')
test_data = pd.read_csv('LD_test.csv')
#train_data = pd.read_csv('NY_train.csv')
#test_data = pd.read_csv('NY_test.csv')

# Model parameters
EPOCHS = 40
BATCH_SIZE = 512

SEED = 42

In [0]:
data = NCFDataset(train=train_data, test=test_data, seed=SEED)

In [0]:
model = NCF (
    n_users=data.n_users, 
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=4,
    layer_sizes=[16,8,4],
    n_epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=1e-3,
    verbose=2,
    seed=SEED
)

In [76]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

Took 235.43479337999997 seconds for training.


In [0]:
predictions = [[row.userID, row.itemID, model.predict(int(row.userID), int(row.itemID))]
               for (_, row) in test_data.iterrows()]

In [78]:
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions

,userID,itemID,prediction
0,14283,6,0.946758
1,14283,6,0.946758
2,11745,9,0.792621
3,11745,6,0.823914
4,7823,7,0.953107
...,...,...,...
39277,8864,870,0.856990
39278,2104,819,0.929929
39279,4631,319,0.511629
39280,4338,638,0.758710


In [0]:
def evaluate_ndcg(y_true, y_score):
    return ndcg_score(np.expand_dims(y_true, axis=0), np.expand_dims(y_score, axis=0), k=5)

In [79]:
print('Testing AUC scores: ', roc_auc_score(test_data['rating'], predictions['prediction']))
print('Testing los_loss scores: ', log_loss(test_data['rating'], predictions['prediction'].astype('float64')))
print("test NDCG@5: ", evaluate_ndcg(test_data['rating'], predictions['prediction']))

Testing AUC scores:  0.9328780050503201
Testing los_loss scores:  3.5445067385980775


test NDCG@5:  0.9999999999999999
